In [28]:
import os
import polars as pl

In [2]:
pl.Config.set_tbl_rows(50)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
slovenske = [
    "S SK 1939",
    "S SR 1939-1945"
]

In [24]:
projevy = pl.read_ndjson(
    'data_raw/schuze/prepis_*.ndjson', 
    ignore_errors=True, 
    schema=pl.Schema(
        {
            'mluvci': pl.String, 
            'soubor': pl.String, 
            'mluvci_id': pl.String, 
            'text': pl.String, 
            'poradi': pl.Int32}
    ))

In [22]:
meta = pl.read_csv(
    'data_raw/schuze/meta_*.csv',
    schema=pl.Schema(
        {
            'komora': pl.String,
            'obdobi': pl.Int32,
            'schuze': pl.Int32,
            'cast': pl.Int32,
            'soubor': pl.String,
            'datum': pl.String,
            'komora_komplet': pl.String,
            'schuze_komplet': pl.String,
            'prepsano': pl.Boolean,
            'autorizovano': pl.Boolean
        }
    )
).with_columns(
    pl.col("datum").str.to_date(format="%Y-%m-%d",exact=False)
)

In [30]:
try:
    os.makedirs("data")
except:
    pass

In [25]:
df = projevy.join(
    meta, on="soubor", how="left"
).filter(
    ~pl.col('komora_komplet').str.contains('SNR')
).filter(
    ~pl.col('komora_komplet').str.contains('NR SR')
).filter(
    ~pl.col('komora_komplet').is_in(slovenske)
).sort(
    by=['datum','poradi']
)

In [32]:
df.write_parquet("data/projevy.parquet", use_pyarrow=True)

In [ ]:
len(df)

In [50]:
len(meta)

102401

In [12]:
bez_data = meta.filter(pl.col("datum").is_null())
print(len(bez_data))

1744


In [14]:
bez_data.filter(pl.col("prepsano") == True)

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano
str,i64,i64,i64,str,date,str,str,bool
"""ssr""",1939,11,2,"""1939ssr_stenprot_011schuz_s011002.htm""",null,"""S SR 1939-1945""","""11. schůze""",true
"""ssr""",1939,126,10,"""1939ssr_stenprot_126schuz_s126010.htm""",null,"""S SR 1939-1945""","""126. schůze""",true
"""ssr""",1939,128,2,"""1939ssr_stenprot_128schuz_s128002.htm""",null,"""S SR 1939-1945""","""128. schůze""",true
"""snr""",1948,2,2,"""1948snr_stenprot_002schuz_s002002.htm""",null,"""SNR 1948-1954""","""2. schůze""",true
"""snr""",1990,7,1,"""1990snr_stenprot_007schuz_s007001.htm""",null,"""SNR 1990-1992""","""7. schůze""",true
"""snr""",1990,7,2,"""1990snr_stenprot_007schuz_s007002.htm""",null,"""SNR 1990-1992""","""7. schůze""",true
"""snr""",1990,7,3,"""1990snr_stenprot_007schuz_s007003.htm""",null,"""SNR 1990-1992""","""7. schůze""",true
"""snr""",1990,7,4,"""1990snr_stenprot_007schuz_s007004.htm""",null,"""SNR 1990-1992""","""7. schůze""",true
"""snr""",1990,7,5,"""1990snr_stenprot_007schuz_s007005.htm""",null,"""SNR 1990-1992""","""7. schůze""",true


In [70]:
bez_data.sort(by='obdobi').tail(30)

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano
str,i64,i64,i64,str,date,str,str,bool
"""ps""",2021,83,53,"""2021ps_stenprot_083schuz_s083053.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,54,"""2021ps_stenprot_083schuz_s083054.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,55,"""2021ps_stenprot_083schuz_s083055.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,56,"""2021ps_stenprot_083schuz_s083056.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,57,"""2021ps_stenprot_083schuz_s083057.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,58,"""2021ps_stenprot_083schuz_s083058.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,59,"""2021ps_stenprot_083schuz_s083059.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,60,"""2021ps_stenprot_083schuz_s083060.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false
"""ps""",2021,83,61,"""2021ps_stenprot_083schuz_s083061.htm""",null,"""PČR, PS 2017-...""","""83. schuze""",false


In [52]:
bez_komory = meta.filter(pl.col("komora_komplet").is_null())
print(len(bez_komory))

1002


In [54]:
bez_obdobi = meta.filter(pl.col("obdobi").is_null())
print(len(bez_obdobi))

0


In [ ]:
bez_data = meta.filter(pl.col("datum").is_null()).sample(30)
print(len(bez_data))

In [40]:
meta.filter(pl.col("datum").is_null()).sample(30)

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano
str,i64,i64,i64,str,date,str,str,bool
"""ps""",2013,23,506,"""2013ps_stenprot_023schuz_s023506.htm""",null,null,"""23. schůze""",false
"""ps""",2017,5,65,"""2017ps_stenprot_005schuz_s005065.htm""",null,null,"""4. schůze""",false
"""ps""",2013,23,26,"""2013ps_stenprot_023schuz_s0232026.htm""",null,null,"""23. schůze""",false
"""snr""",1990,24,48,"""1990snr_stenprot_024schuz_s024048.htm""",null,"""SNR 1990-1992""","""24. schůze""",true
"""ps""",1996,1,8,"""1996ps_psse_stenprot_001schuz_s001008.htm""",null,"""PČR 1996-1998""","""1. schůze""",true
"""ps""",2017,5,189,"""2017ps_stenprot_005schuz_s005189.htm""",null,null,"""4. schůze""",false
"""snr""",1990,10,8,"""1990snr_stenprot_010schuz_s010008.htm""",null,"""SNR 1990-1992""","""10. schůze""",true
"""ps""",2013,23,430,"""2013ps_stenprot_023schuz_s023430.htm""",null,null,"""23. schůze""",false
"""snr""",1990,18,35,"""1990snr_stenprot_018schuz_s018035.htm""",null,"""SNR 1990-1992""","""18. schůze""",true


In [7]:
import altair as alt

In [8]:
alt.Chart(
    df.filter(~pl.col('datum').is_null()).group_by_dynamic(index_column="datum", every="1y").agg(pl.col('text').len()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("text:Q")
)

alt.Chart(...)

In [134]:
df.filter(pl.col('datum').dt.year() == 1976)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano
str,str,str,str,i64,str,i64,i64,i64,date,str,str,bool
"""""","""1971fs_slsn_023schuz_s023009""",null,"""Předsedající předseda SN D. Ha…",1,"""fs""",1971,23,9,1976-06-22,"""FS ČSSR 1971-1976""","""23. schůze""",true
"""""","""1976fs_slsn_002schuz_s002005""",null,"""Poslanec SL prof. JUDr. Z. Češ…",1,"""fs""",1976,2,5,1976-12-14,"""FS ČSSR 1976-1980""","""2. schůze""",true
"""""","""1976fs_slsn_002schuz_s002022""",null,"""Poslanec SL J. Prchal: Vážený …",1,"""fs""",1976,2,22,1976-12-15,"""FS ČSSR 1976-1980""","""2. schůze""",true
"""""","""1976fs_slsn_002schuz_s002025""",null,"""Předsedající předseda SN prof.…",1,"""fs""",1976,2,25,1976-12-15,"""FS ČSSR 1976-1980""","""2. schůze""",true


In [12]:
meta = pl.read_csv(
        'data_raw/schuze/*.csv'
    ).with_columns(
        pl.col("datum").str.to_date()
    ).sort(by="datum")

In [110]:
meta.columns

['komora',
 'obdobi',
 'schuze',
 'cast',
 'soubor',
 'datum',
 'komora_komplet',
 'schuze_komplet',
 'prepsano']

In [118]:
meta.sample(10)

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano
str,i64,i64,i64,str,date,str,str,bool
"""ps""",2021,55,170,"""2021ps_055schuz_s055170""",2023-03-01,"""PČR, PS 2021-...""","""55. schůze""",true
"""ps""",2017,30,239,"""2017ps_030schuz_s030239""",2019-06-07,"""PČR, PS 2017-2021""","""30. schůze""",true
"""ps""",2013,42,233,"""2013ps_042schuz_s042233""",2016-03-10,"""PČR, PS 2013-2017""","""42. schůze""",true
"""ps""",2006,28,241,"""2006ps_028schuz_s028241""",2008-03-25,"""PČR, PS 2006-2010""","""28. schůze""",true
"""nr""",1998,5,7,"""1998nr_005schuz_s005007""",1998-11-20,"""NR SR, 1998-2002""","""5. schůze""",true
"""ps""",2017,101,1,"""2017ps_101schuz_s101001""",2021-05-05,"""PČR, PS 2017-2021""","""101. schůze""",true
"""ps""",2021,119,228,"""2021ps_119schuz_s119228""",2024-12-03,"""PČR, PS 2021-...""","""119. schůze""",true
"""ns""",1920,27,9,"""1920ns_ps_027schuz_s027009""",1920-11-30,"""NS RČS 1920-1925""","""27. schůze""",true
"""uns""",1946,92,2,"""1946uns_092schuz_s092002""",1948-02-03,"""Ústavodárné NS RČS 1946-1948""","""92. schůze""",true


In [13]:
alt.Chart(
    meta.filter(~pl.col('datum').is_null()).group_by_dynamic(index_column="datum", every="1y").agg(pl.col('soubor').len()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("soubor:Q")
)

alt.Chart(...)

In [124]:
alt.Chart(
    meta.filter(~pl.col('datum').is_null()).filter(pl.col("prepsano") == True).group_by_dynamic(index_column="datum", every="1y").agg(pl.col('soubor').len()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("soubor:Q")
)

alt.Chart(...)

In [48]:
df.filter(~pl.col("soubor").is_null()).sort(by='soubor')

mluvci,soubor,mluvci_id,text,poradi
str,str,str,str,i64
"""""","""1861skc_012schuz_s012005""",null,"""Dr. Taschek m. p., Fric m. p a…",1
"""""","""1861skc_012schuz_s012016""",null,"""Mit Beschluß vom 18. d. Mts., …",1
"""""","""1918ns_ps_014schuz_s014001""",null,"""Tisk 277. Návrh členů N. S. dr…",1
"""(pokračuje): Ještě bych chtěl …","""1918ns_ps_016schuz_s016008""",null,"""Vy máte pro to zvláštní formul…",2
"""(pokračuje): Velectění pánové,…","""1918ns_ps_025schuz_s025002""",null,"""Jest známo, že v dobách rakous…",2
…,…,…,…,…
"""Poslanec Tomáš Müller""","""2021ps_140schuz_s140048""","""id=6772""","""Děkuji za slovo. Kolegyně, kol…",2
"""Předsedkyně PSP Markéta Pekaro…","""2021ps_140schuz_s140048""","""id=6147""","""A já také děkuji. Nyní otevírá…",3
"""Poslanec Rudolf Salvetr""","""2021ps_140schuz_s140048""","""id=6744""","""Děkuji za slovo, paní předsedk…",4


In [46]:
df.filter(pl.col('text').str.contains('Brno'))

mluvci,soubor,mluvci_id,text,poradi
str,str,str,str,i64
""" Zpravodaj poslanec Ferdinand …",null,null,"""Zpravodaj poslanec Ferdinand J…",7
""" Zpravodaj posl. dr Slávik""",null,null,"""Zpravodaj posl. dr Slávik: Slá…",3
"""(pokračuje): Předloha nemluví …","""1929ns_se_075schuz_s075009""",null,"""Je význačno, že stát, vláda sv…",2
"""(pokračuje): Z toho připadá: n…","""1929ns_se_298schuz_s298012""",null,"""Uvádím podrobně úkoly a dotová…",2
"""Poslanec František Volejník""",null,null,"""Poslanec František Volejník: S…",2
…,…,…,…,…
"""Místopředseda vlády a ministr …","""2021ps_094schuz_s094218""","""https://www.vlada.cz/cz/clenov…","""Já bych chtěl zareagovat vlast…",4
"""Poslanec Andrej Babiš""","""2021ps_098schuz_s098075""","""id=6150""","""Pane ministře, já vám závidím,…",2
"""Poslankyně Renáta Zajíčková""","""2021ps_115schuz_s115045""","""id=6793""","""Dobrý večer. Svým příspěvkem v…",4


In [40]:
df.filter(~pl.col('soubor').is_null())

mluvci,soubor,mluvci_id,text,poradi
str,str,str,str,i64
"""""","""1861skc_012schuz_s012005""",null,"""Dr. Taschek m. p., Fric m. p a…",1
"""""","""1861skc_012schuz_s012016""",null,"""Mit Beschluß vom 18. d. Mts., …",1
"""""","""1918ns_ps_014schuz_s014001""",null,"""Tisk 277. Návrh členů N. S. dr…",1
"""(pokračuje): Ještě bych chtěl …","""1918ns_ps_016schuz_s016008""",null,"""Vy máte pro to zvláštní formul…",2
"""(pokračuje): Velectění pánové,…","""1918ns_ps_025schuz_s025002""",null,"""Jest známo, že v dobách rakous…",2
…,…,…,…,…
"""Poslanec Tomáš Müller""","""2021ps_140schuz_s140048""","""id=6772""","""Děkuji za slovo. Kolegyně, kol…",2
"""Předsedkyně PSP Markéta Pekaro…","""2021ps_140schuz_s140048""","""id=6147""","""A já také děkuji. Nyní otevírá…",3
"""Poslanec Rudolf Salvetr""","""2021ps_140schuz_s140048""","""id=6744""","""Děkuji za slovo, paní předsedk…",4


In [42]:
df.filter(~pl.col('mluvci_id').is_null())

mluvci,soubor,mluvci_id,text,poradi
str,str,str,str,i64
"""hlasování č. 165""","""1996ps_011schuz_s011067""","""G=9196""","""tí tohoto návrhu, ať zvedne ru…",26
"""Poslanec Vojtěch Filip""","""1998ps_007schuz_s007118""","""P=452""","""Děkuji, pane místopředsedo. Pa…",4
"""Místopředseda PSP Ivan Langer""","""1998ps_007schuz_s007118""","""P=516""","""Děkuji předsedovi poslaneckého…",5
"""Místopředseda PSP Ivan Langer""","""1998ps_007schuz_s007118""","""P=516""","""Děkuji zástupci navrhovatelů. …",7
"""Poslanec Marek Benda""","""1998ps_007schuz_s007118""","""P=425""","""Vážený pane místopředsedo, dám…",8
…,…,…,…,…
"""Poslanec Tomáš Müller""","""2021ps_140schuz_s140048""","""id=6772""","""Děkuji za slovo. Kolegyně, kol…",2
"""Předsedkyně PSP Markéta Pekaro…","""2021ps_140schuz_s140048""","""id=6147""","""A já také děkuji. Nyní otevírá…",3
"""Poslanec Rudolf Salvetr""","""2021ps_140schuz_s140048""","""id=6744""","""Děkuji za slovo, paní předsedk…",4


In [52]:
meta = 

In [54]:
df = df

In [84]:
df.filter(pl.col("text").str.contains("[Kk]ašpárek")).with_columns(pl.col("datum").dt.year().alias("rok")).group_by("rok").len().sort(by="rok")

rok,len
i32,u32
1994,1
2024,1


In [82]:
df.filter(pl.col("text").str.contains("láska")).with_columns(pl.col("datum").dt.year().alias("rok")).group_by("rok").len().sort(by="rok")

rok,len
i32,u32
null,9
1919,1
1921,1
1926,2
1933,1
…,…
1993,1
2004,1
2013,1


In [70]:
df.with_columns(pl.col("datum").dt.year().alias("rok")).group_by("rok").len().sort(by="rok")

rok,len
i32,u32
null,2498
1861,2
1919,21
1920,25
1921,15
…,…
2021,1886
2022,1887
2023,2566


In [80]:
df.with_columns(pl.col("datum").dt.year().alias("rok")).group_by("rok").agg(pl.col("mluvci_id").unique().len()).sort(by="rok").tail(30)

rok,mluvci_id
i32,u32
1996,1
1997,2
1998,15
1999,197
2000,211
…,…
2021,201
2022,183
2023,217


In [ ]:
df.with_columns(pl.col("datum").dt.year().alias("rok")).group_by("rok").agg(pl.col("mluvci_id").unique().len()).sort(by="rok").tail(30)